In [19]:
import sqlite3

# 데이터베이스 연결
db_path = r"C:\Users\Playdata\Desktop\만개레시피찐찐찐찐찐 최종\SKN06-FINAL-6Team\data\funstaurant.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 데이터베이스 내 테이블 목록 조회
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 테이블 목록 출력
tables


[('funstaurant',), ('sqlite_sequence',)]

In [20]:
# 'funstaurant' 테이블의 컬럼 정보 조회
cursor.execute("PRAGMA table_info(funstaurant);")
columns = cursor.fetchall()

# 컬럼 정보 출력
columns


[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 1, None, 0),
 (2, 'photo', 'TEXT', 0, None, 0),
 (3, 'ingredients', 'TEXT', 1, None, 0),
 (4, 'recipe', 'TEXT', 1, None, 0)]

In [21]:
# 'funstaurant' 테이블에서 일부 데이터 조회
cursor.execute("SELECT * FROM funstaurant;")
sample_data = cursor.fetchall()

# 데이터 출력
import pandas as pd

df = pd.DataFrame(sample_data, columns=["id", "name", "photo", "ingredients", "recipe"])
print(df)

        id                                       name  \
0        1                  [257회] 김재중 - '빨간왕갈비탕' 레시피   
1        2                    [257회] 장신영 - '김 구이' 레시피   
2        3                    [257회] 장신영 - '동그랑땡' 레시피   
3        4             [257회] 장신영 - '메추리알 짜장 장조림' 레시피   
4        5                    [257회] 장신영 - '감자조림' 레시피   
...    ...                                        ...   
1811  1812  [4회] 김나영 - &lt;먹더조이 – 곤드레 두유 아란치니&gt; 레시피   
1812  1813      [4회] 김나영 - &lt;먹더조이 - 한 입 핫도그&gt; 레시피   
1813  1814     [3회] 진세연 - &lt;감자크림 명란 떡볶이&gt; 레시피-떡볶이   
1814  1815            [3회] 정혜영 - &lt;아보카도 밥버거&gt; 레시피   
1815  1816           [3회] 정혜영 -  &lt;하와이안 주먹밥&gt; 레시피   

                                                  photo  \
0     ['https://pbbsres.kbs.co.kr/t2019-0286-04-2963...   
1     ['https://pbbsres.kbs.co.kr/t2019-0286-04-2963...   
2     ['https://pbbsres.kbs.co.kr/t2019-0286-04-2963...   
3     ['https://pbbsres.kbs.co.kr/t2019-0286-04-2963...   
4     ['https://pbbs

- 요리명 정리: [257회] 등의 불필요한 회차 정보 제거
- 사진 URL 정리: 리스트 형태에서 첫 번째 URL만 남김
- 재료 리스트 변환: ingredients를 개별 재료 항목으로 리스트화
- 조리법 정리: 개행 문자 및 불필요한 공백 제거

In [22]:
import re

# 회차 및 불필요한 문자 제거 함수
def clean_name(name):
    if name:
        # 회차 제거 (예: "[257회] " 제거)
        name = re.sub(r"\[\d+회\]\s*", "", name)
        # HTML 엔티티 제거 (예: "&lt;", "&gt;")
        name = re.sub(r"&lt;|&gt;", "", name)
        return name.strip()
    return name

# name 컬럼 정리 적용
df["clean_name"] = df["name"].apply(clean_name)

# 정리된 데이터 컬럼 반영
df = df[["id", "clean_name", "ingredients", "recipe"]]
df.rename(columns={"clean_name": "name"}, inplace=True)

print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814     진세연 - 감자크림 명란 떡볶이 레시피-떡볶이   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     \n갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 ...   
1                                 \n돌김, 들기름, 식용유, 맛소금\n   
2     \n돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개\n다진...   
3     \n메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200m...   
4     \n감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, ...   
...                                                 ...   
1811  \n양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200...   

In [23]:
# '레시피' 뒤에 있는 모든 텍스트 제거 함수 (요리명 유지)
def clean_recipe_suffix(name):
    if name:
        return re.sub(r"(레시피).*", r"\1", name).strip()  # '레시피' 이후 모든 텍스트 삭제
    return name

# name 컬럼 정리 적용
df["clean_name"] =df["name"].apply(clean_recipe_suffix)

# 정리된 데이터 컬럼 반영
df = df[["id", "clean_name", "ingredients", "recipe"]]
df.rename(columns={"clean_name": "name"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     \n갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 ...   
1                                 \n돌김, 들기름, 식용유, 맛소금\n   
2     \n돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개\n다진...   
3     \n메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200m...   
4     \n감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, ...   
...                                                 ...   
1811  \n양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200...   

In [24]:
# 재료 목록 전처리 함수
def clean_ingredients(ingredients):
    if ingredients:
        # 개행 문자 및 불필요한 공백 제거
        ingredients = ingredients.replace("\n", " ").strip()
        # 특수 문자 정리 (예: 연속된 공백 제거)
        ingredients = re.sub(r"\s+", " ", ingredients)
        return ingredients
    return ingredients

# ingredients 컬럼 정리 적용
df["clean_ingredients"] = df["ingredients"].apply(clean_ingredients)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "clean_ingredients", "recipe"]]
df.rename(columns={"clean_ingredients": "ingredients"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [25]:
# 특수 문자 제거 함수
def remove_special_chars(text):
    if text:
        # 특수 문자 제거 (예: ■, ★, ※ 등)
        text = re.sub(r"[■★※●◆◇■□▽▼○◎]", "", text)
        # 불필요한 공백 정리
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return text

# ingredients 컬럼에서 특수 문자 제거 적용
df["clean_ingredients"] = df["ingredients"].apply(remove_special_chars)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "clean_ingredients", "recipe"]]
df.rename(columns={"clean_ingredients": "ingredients"}, inplace=True)

# 정리된 데이터 확인
print(df)


        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [26]:
# 특수 문자 및 불필요한 기호 제거 함수 (추가 기호 포함)
def clean_recipe_text(text):
    if text:
        # 기본 특수 문자 및 추가 기호 제거 (!, ~, *, @ 등)
        text = re.sub(r"[■★※●◆◇■□ㅁ♥▽▼○◎!~*@#%^&+=<>?|]", "", text)
        # 불필요한 공백 정리
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return text

# recipe 컬럼에서 특수 문자 및 추가 기호 제거 적용
df["clean_recipe"] = df["recipe"].apply(clean_recipe_text)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)


        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [27]:
# 레시피 번호 매기기 방식 정리 함수 (0. 삭제 + 이모티콘 번호 변경)
def refine_numbering_v2(text):
    if text:
        # "0." 삭제 (특정 패턴 포함 가능)
        text = re.sub(r"\b0\.\s*", "", text)
        # ①, ② 같은 이모티콘을 1., 2. 형식으로 변경
        text = re.sub(r"[①❶]", "1.", text)
        text = re.sub(r"[②❷]", "2.", text)
        text = re.sub(r"[③❸]", "3.", text)
        text = re.sub(r"[④❹]", "4.", text)
        text = re.sub(r"[⑤❺]", "5.", text)
        text = re.sub(r"[⑥❻]", "6.", text)
        text = re.sub(r"[⑦❼]", "7.", text)
        text = re.sub(r"[⑧❽]", "8.", text)
        text = re.sub(r"[⑨❾]", "9.", text)
        text = re.sub(r"[⑩❿]", "10.", text)
        # 불필요한 공백 정리
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return text

# recipe 컬럼에서 번호 매기기 방식 수정 적용
df["clean_recipe"] = df["recipe"].apply(refine_numbering_v2)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [ ]:
def remove_instagram_text(text):
    if text:
        return text.replace("레시피는 편스토랑 인스타그램에서도 확인할 수 있습니다 funstaurant_kbs", "").strip()
    return text


# recipe 컬럼에서 번호 매기기 방식 수정 적용
df["clean_recipe"] = df["recipe"].apply(remove_instagram_text)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [31]:
def remove_instagram_text2(text):
    if text:
        return text.replace("레시피는 편스토랑 인스타그램에도 확인할 수 있습니다 funstaurant_kbs", "").strip()
    return text


# recipe 컬럼에서 번호 매기기 방식 수정 적용
df["clean_recipe"] = df["recipe"].apply(remove_instagram_text2)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [ ]:
def remove_instagram_text3(text):
    if text:
        return text.replace("레시피는 편스토랑 인스타그램 에서도 확인할 수 있습니다 funstaurant_kbs", "").strip()
    return text


# recipe 컬럼에서 번호 매기기 방식 수정 적용
df["clean_recipe"] = df["recipe"].apply(remove_instagram_text3)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)

        id                          name  \
0        1            김재중 - '빨간왕갈비탕' 레시피   
1        2              장신영 - '김 구이' 레시피   
2        3              장신영 - '동그랑땡' 레시피   
3        4       장신영 - '메추리알 짜장 장조림' 레시피   
4        5              장신영 - '감자조림' 레시피   
...    ...                           ...   
1811  1812  김나영 - 먹더조이 – 곤드레 두유 아란치니 레시피   
1812  1813      김나영 - 먹더조이 - 한 입 핫도그 레시피   
1813  1814         진세연 - 감자크림 명란 떡볶이 레시피   
1814  1815            정혜영 - 아보카도 밥버거 레시피   
1815  1816           정혜영 -  하와이안 주먹밥 레시피   

                                            ingredients  \
0     갈비 8대, 양파 2개, 대파 2대, 무 반 통, 생강 2톨, 인삼 2개, 밤 5알...   
1                                     돌김, 들기름, 식용유, 맛소금   
2     돼지고기 다짐육 200g, 두부 1/2모, 대파 1/2개, 당근 1/3개 다진마늘 ...   
3     메추리알 20개, 비엔나소시지 70g, 짜장가루 3T, 설탕 1T, 물 200ml,...   
4     감자 2개, 마늘종, 물엿 2T, 다진마늘 1T, 식용유 2T, 고추장 1T, 고춧...   
...                                                 ...   
1811  양송이버섯 3개, 양파 1/4개, 카놀라유, 곤드레밥 1팩, 무가당 두유 200ml...   

In [ ]:
def remove_instagram_text4(text):
    if text:
        return text.replace("레시피는 편스토랑 인스타그램에서도 확인할 수 있습니다", "").strip()
    return text


# recipe 컬럼에서 번호 매기기 방식 수정 적용
df["clean_recipe"] = df["recipe"].apply(remove_instagram_text4)

# 정리된 데이터 컬럼 반영
df = df[["id", "name", "ingredients", "clean_recipe"]]
df.rename(columns={"clean_recipe": "recipe"}, inplace=True)

# 정리된 데이터 확인
print(df)

In [34]:
# 전처리된 데이터를 새로운 SQLite DB 파일로 저장하는 코드

# 새로운 데이터베이스 파일 경로 설정
new_db_path = r"C:\Users\Playdata\Desktop\만개레시피찐찐찐찐찐 최종\SKN06-FINAL-6Team\data\funstaurant_1차.db"

# SQLite 데이터베이스에 저장
conn_new = sqlite3.connect(new_db_path)
df.to_sql("funstaurant_cleaned", conn_new, if_exists="replace", index=False)
conn_new.close()

# 저장된 DB 파일 경로 출력
new_db_path


'C:\\Users\\Playdata\\Desktop\\만개레시피찐찐찐찐찐 최종\\SKN06-FINAL-6Team\\data\\funstaurant_1차.db'